# Neural Code Final Portfolio - Component 2

This notebook gives a more detailed exposition of the analyses performed in Courellis et al. (2024).

## Preliminaries

In [1]:
# Required packages
import os
import numpy as np
import pandas as pd
from scipy.io import loadmat
import helper_functions as F

# Paths to files
FOLDER_PATH = "2024courellis/"
neu_mat_path = os.path.join(FOLDER_PATH, "data/neu.mat")

## Load and inspect dataset


Each row of the `neu_data` dataset corresponds to a single neuron. 

* `cellinfo`: index of brain region that the cell is in;
* `sessionID`: unique identifier for each recording session, during which paricipants performed a serial reversal-learning task;
* `array`: a subtable of data for each neuron that contains information for all trials in the session.

TO DO: what is a trial?

In [2]:
neu_data = pd.read_json("neu.json")
neu_data["n_trials"] = neu_data["array"].apply(lambda x : len(x))
neu_data

,array,cellinfo,sessionID,n_trials
0,"[{'block_nr': 1, 'iscorrect': True, 'stim_id':...",2,P61CS_1,320
1,"[{'block_nr': 1, 'iscorrect': True, 'stim_id':...",2,P61CS_1,320
2,"[{'block_nr': 1, 'iscorrect': True, 'stim_id':...",2,P61CS_1,320
3,"[{'block_nr': 1, 'iscorrect': True, 'stim_id':...",4,P61CS_1,320
4,"[{'block_nr': 1, 'iscorrect': True, 'stim_id':...",4,P61CS_1,320
...,...,...,...,...
2689,"[{'block_nr': 1, 'iscorrect': False, 'stim_id'...",3,TWH172_2,240
2690,"[{'block_nr': 1, 'iscorrect': False, 'stim_id'...",3,TWH172_2,240
2691,"[{'block_nr': 1, 'iscorrect': False, 'stim_id'...",3,TWH172_2,240
2692,"[{'block_nr': 1, 'iscorrect': False, 'stim_id'...",3,TWH172_2,240


In [3]:
neu_data["n_trials"].value_counts()

n_trials
240    2309
260     119
200     105
180      65
320      57
280      39
Name: count, dtype: int64

In [4]:
type(neu_data["n_trials"].unique())

numpy.ndarray

In [5]:
def get_cell_array(
    neu_data : pd.DataFrame,
    cell_idx : int
 ) -> pd.DataFrame:
    cell_array_dict = {}
    for row_dict in neu_data["array"][cell_idx]:
        for key, value in row_dict.items():
            if key not in cell_array_dict:
                cell_array_dict[key] = [value]
            else:
                cell_array_dict[key].append(value)
    cell_array_data = pd.DataFrame(cell_array_dict)
    return cell_array_data

cell_array_example = get_cell_array(neu_data=neu_data, cell_idx=0)
cell_array_example

,block_nr,iscorrect,stim_id,context,reward,response,trial_nr,fr_stim,fr_base
0,1,True,2,2,25,0,1,0,0.000000
1,1,True,4,2,5,0,2,0,0.000000
2,1,True,1,2,5,1,3,2,0.000000
3,1,True,2,2,25,0,4,1,0.909091
4,1,True,4,2,5,0,5,0,0.909091
...,...,...,...,...,...,...,...,...,...
315,10,True,3,1,5,0,19,3,0.909091
316,10,True,3,1,5,0,20,1,0.000000
317,10,True,2,1,25,1,21,1,1.818182
318,10,True,2,1,25,1,22,0,1.818182


## Perform geometric analysis on balanced dichotomies

We have the following parameters for running the analysis:

In [ ]:
n_resample = 5
n_perm_inner = 1
n_samples = [15, 15, 15, 15, 15, 10]

### Step 1: Aggregate neurons by area

Individual neuron data was combined across all participants to form a single "pseudo-population."

In [ ]:
area_order = ['HPC','vmPFC','AMY','dACC','preSMA','VTC']
idx_order = [3, 1, 4, 2, 9, 12]
cellinfo = neu_data.cellinfo.to_numpy()

cell_area_groups = {}
for i, area in enumerate(area_order):
    idx = np.where(cellinfo == idx_order[i])[0]
    cell_area_groups[area_order[i]] = idx

print("Distribution of neurons across brain areas:")
for key, value in cell_area_groups.items():
    print(f"{key}: {len(value)}")

Distribution of neurons across brain areas:
HPC: 494
vmPFC: 889
AMY: 269
dACC: 310
preSMA: 463
VTC: 269


### Step 2: Define metrics for assessment

#### Balanced dichotomies

In [ ]:
import helper_functions as F
import itertools


def make_variable_groups(
    cell_data : pd.DataFrame,
    var_names : list[str]
):
    """
    Create groups of all possible combinations of values for the given 
    variables.

    Parameters
    ----------
    cell_data : pd.DataFrame
        Data for a single neuron, where each row represents one trial.
    var_names : list[str]
        List of DataFrame columns to consider.

    Returns:
    groups : pd.DataFrame
        DataFrame where each row represents a trial and each column represents 
        membership in a group.
    """
    unique_values = [cell_data[var].unique() for var in var_names]
    combinations = list(itertools.product(*unique_values))
    group_names = [
        "_".join([f"{var}_{value}" for var, value in zip(var_names, combo)])
        for combo in combinations
    ]
    groups = pd.DataFrame(np.zeros(
        (len(cell_data), len(combinations)), dtype=bool
    ), columns=group_names)
    for i, combo in enumerate(combinations):
        # Set combination membership for all trials in a single column
        condition = np.all([
            cell_data[var] == value for var, value in zip(var_names, combo)
        ], axis=0)
        groups[group_names[i]] = condition
    return groups



,reward_25_context_2,reward_25_context_1,reward_5_context_2,reward_5_context_1,reward_0_context_2,reward_0_context_1
0,True,False,False,False,False,False
1,False,False,True,False,False,False
2,False,False,True,False,False,False
3,True,False,False,False,False,False
4,False,False,True,False,False,False
...,...,...,...,...,...,...
315,False,False,False,True,False,False
316,False,False,False,True,False,False
317,False,True,False,False,False,False
318,False,True,False,False,False,False


In [ ]:
def construct_regressors(
    neu_data : pd.DataFrame,
    thr : int,
    select : list
):
    """
    Method for balancing neuron counts between inference absent (ia) and 
    inference present (ip) groups.
    
    Parameters
    ----------
    neu_data : pd.DataFrame
        DataFrame with each row correpsonding to all trial data for a single 
        neuron.
    thr : int
        Minimum number of correct trials of each type to retain neurons.
    select : list
        Neuron indices to include.

    Returns
    -------
    """
    group_avgs = []
    for i in select:
        # Select only correct trials for given neuron
        cell_data = get_cell_array(neu_data=neu_data, cell_idx=i)
        valid_data = cell_data[cell_data["iscorrect"] == True]
        
        # Make trial-level labels according to binary task variables
        groups = make_variable_groups(
            cell_data=valid_data,
            var_names=["reward", "response", "context"]
        )     

        firing_rate = valid_data["fr_stim"].values
        group_avg = []
        for combo in groups.columns:
            # Get boolean mask for inclusion in current group
            group_firing_rate = firing_rate[groups[combo]]
            # Check that neuron has enough samples of the current type
            if len(group_firing_rate) > thr:
                group_avg.append(group_firing_rate.mean())
        # Only append data for neurons with enough trials in all types
        if len(group_avg) == len(groups.columns):
            group_avgs.append(group_avg)
    return pd.DataFrame(np.array(group_avgs), columns=groups.columns)

construct_regressors(neu_data, thr=30, select=[i for i in range(100)])

,reward_25_response_0_context_2,reward_25_response_0_context_1,reward_25_response_1_context_2,reward_25_response_1_context_1,reward_5_response_0_context_2,reward_5_response_0_context_1,reward_5_response_1_context_2,reward_5_response_1_context_1
0,1.604651,1.441176,1.967742,1.648649,1.777778,1.888889,1.659574,2.026316
1,2.418605,2.500000,2.677419,2.540541,2.194444,2.361111,2.468085,2.394737
2,0.279070,0.558824,0.580645,0.270270,0.416667,0.361111,0.468085,0.447368
3,0.558140,1.852941,1.225806,1.216216,0.750000,1.638889,0.468085,1.236842
4,5.953488,5.617647,6.516129,5.405405,6.222222,5.722222,5.446809,6.236842
5,0.395349,0.294118,0.354839,0.270270,0.222222,0.416667,0.191489,0.289474
6,8.790698,10.794118,8.774194,9.000000,9.527778,10.027778,8.531915,8.973684
7,1.279070,1.176471,0.935484,0.864865,0.694444,0.861111,1.021277,1.289474
8,1.023256,0.970588,1.032258,0.756757,0.833333,0.916667,0.872340,1.157895
9,0.744186,0.735294,0.741935,0.513514,0.361111,0.555556,0.744681,0.789474


#### Shattering dimensionality

In [ ]:
def shattering_dimensionality(
    avg_array,
    n_iter : int,
    sample_thr : int
):
    """
    Method for performing shattering dimensionality analysis 
    for a group of cells.

    Parameters
    ----------
    avg_array : np.array
        Cell array of neurons that exceed the trial count threshold.
    n_iter : int
        Number of iterations of bootstrap re-sampling to perform.
    sample_thr : int
        Number of trials of each condition to sample.
    """

In [6]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

def sd(avg_array, nr_iterations, nr_samples):
    """
    Perform shattering dimensionality analysis for a group of cells.

    Parameters:
    avg_array (list): List of neurons, with firing rates grouped hierarchically by trial type.
    nr_iterations (int): Number of bootstrap iterations to perform.
    nr_samples (int): Number of trials of each condition to sample.

    Returns:
    tuple: perf, boot, nr_cells
    perf (ndarray): Decoding performance for every dichotomy across iterations.
    boot (ndarray): Null distribution constructed using trial-label shuffle.
    nr_cells (int): Number of cells used in analysis.
    """
    PosSet, NegSet = define_sets()
    nr_cells = len(avg_array)
    nr_folds = 5

    perf = np.full((PosSet.shape[0], nr_iterations), np.nan)
    boot = np.full((PosSet.shape[0], nr_iterations), np.nan)

    for i in range(PosSet.shape[0]):
        gr_1 = PosSet[i]
        gr_2 = NegSet[i]

        for j in range(nr_iterations):
            training, testing = sample_from_data(avg_array, 0, nr_samples)
            training_data, _, labels_train, _ = prep_regressors(training, testing, gr_1, gr_2)

            # Normalize and remove NaN features
            scaler = StandardScaler()
            training_data = scaler.fit_transform(training_data)
            training_data = training_data[:, ~np.isnan(training_data).any(axis=0)]

            # Fit logistic regression model (equivalent to linear SVM)
            clf = LogisticRegression(max_iter=1000, solver='liblinear')
            y_hat = cross_val_predict(clf, training_data, labels_train, cv=nr_folds)
            perf[i, j] = accuracy_score(labels_train, y_hat)

            # Compute null distribution (shuffle labels)
            shuffled_labels = shuffle(labels_train)
            y_hat_null = cross_val_predict(clf, training_data, shuffled_labels, cv=nr_folds)
            boot[i, j] = accuracy_score(shuffled_labels, y_hat_null)

        print(f"Finished dichotomy {i + 1}")

    return perf, boot, nr_cells



def sample_from_data(array, test_samples, train_samples):
    """
    Sample data for training and testing.

    Returns:
    tuple: training, testing (both lists)
    """
    testing = [None] * len(array)
    training = [None] * len(array)

    for i in range(len(array)):
        testing[i] = []
        training[i] = []

        for trials in array[i]:
            test_indices = np.random.choice(len(trials), test_samples, replace=False)
            testing[i].append(trials[test_indices])

            remaining_trials = np.delete(trials, test_indices)
            train_indices = np.random.choice(len(remaining_trials), train_samples, replace=False)
            training[i].append(remaining_trials[train_indices])

    return training, testing

def prep_regressors(training, testing, gr_1, gr_2):
    """
    Prepare regressors for training and testing.

    Returns:
    tuple: training, testing, labels_train, labels_test
    """
    training_1 = np.vstack([np.vstack(training[i][gr_1]) for i in range(len(training))])
    training_2 = np.vstack([np.vstack(training[i][gr_2]) for i in range(len(training))])

    labels_train = np.concatenate([np.ones(len(training_1)), -1 * np.ones(len(training_2))])

    testing_1 = np.vstack([np.vstack(testing[i][gr_1]) for i in range(len(testing))])
    testing_2 = np.vstack([np.vstack(testing[i][gr_2]) for i in range(len(testing))])

    labels_test = np.concatenate([np.ones(len(testing_1)), -1 * np.ones(len(testing_2))])

    training = np.vstack([training_1, training_2])
    testing = np.vstack([testing_1, testing_2])

    return training, testing, labels_train, labels_test

### Step 3: Run geometric analyses

In [ ]:
import numpy as np
import pandas as pd

# Assuming neu_data is a pandas DataFrame containing 'array' and 'sessions'
# and relevant functions like construct_regressors, sd, ccgp, ps are implemented.

# Initialize containers
sd_ = [[], []]  # To store results for two conditions (inference absent, present)
sd_boot = [[], []]
ccgp_ = [[], []]
ccgp_boot = [[], []]
ps_ = [[], []]
ps_boot = [[], []]

# Example of cell_groups (brain areas)
cell_groups = [idx_hpc, idx_pfc, idx_amy, idx_acc, idx_sma, idx_vtc]

# Loop over each brain area
for i, area_indices in enumerate(cell_groups):
    # For inference absent sessions
    idx_current = np.intersect1d(area_indices, np.where(np.isin(sessions, inference_absent))[0])

    for _ in range(n_resample):
        avg_array = construct_regressors(neu, n_samples[i], idx_current)
        
        t_1, t_2 = sd(avg_array, n_perm_inner, n_samples[i])
        sd_[i].append(t_1)
        sd_boot[i].append(t_2)
        
        ccgp_result = ccgp(avg_array, n_perm_inner, False, n_samples[i])
        ccgp_[i].append(ccgp_result)
        
        ccgp_boot_result = ccgp(avg_array, n_perm_inner, True, n_samples[i])
        ccgp_boot[i].append(ccgp_boot_result)
        
        ps_result = ps(avg_array, n_perm_inner, False)
        ps_[i].append(ps_result)
        
        ps_boot_result = ps(avg_array, n_perm_inner, True)
        ps_boot[i].append(ps_boot_result)

    # Repeat for inference present sessions
    idx_current = np.intersect1d(area_indices, np.where(np.isin(sessions, inference_present))[0])

    for _ in range(n_resample):
        avg_array = construct_regressors(neu, n_samples[i], idx_current)
        
        t_1, t_2 = sd(avg_array, n_perm_inner, n_samples[i])
        sd_[i].append(t_1)
        sd_boot[i].append(t_2)
        
        ccgp_result = ccgp(avg_array, n_perm_inner, False, n_samples[i])
        ccgp_[i].append(ccgp_result)
        
        ccgp_boot_result = ccgp(avg_array, n_perm_inner, True, n_samples[i])
        ccgp_boot[i].append(ccgp_boot_result)
        
        ps_result = ps(avg_array, n_perm_inner, False)
        ps_[i].append(ps_result)
        
        ps_boot_result = ps(avg_array, n_perm_inner, True)
        ps_boot[i].append(ps_boot_result)
